Jaseung Ku, 07/01/2020

# Run this once

In [1]:
%matplotlib 
import matplotlib.pyplot as plt
import numpy as np
from qutip import *

def compute_transmon_f01(Ej, Ecm, ng=0, N=10):
    """
    Compute f01 qubit frequency of transmon
    Args:
        Ej, Ec : in GHz
        ng: unitless gate charge
        N: number of charge basis = 2N + 1
    Return:
        f01 qubit frequency
    """
    H = Qobj(np.diag(4 * Ec * (np.arange(-N,N+1)-ng)**2) 
             + 0.5 * Ej * (np.diag(-np.ones(2*N), 1) + np.diag(-np.ones(2*N), -1)))
    eigenvalues = H.eigenenergies()
    f01 = eigenvalues[1] - eigenvalues[0]
    
    return f01

Using matplotlib backend: TkAgg


# Two-level qubit + resonator

In [2]:
def compute_energies_two_level_qubit(wc, wa, g, N=10, use_rwa=False):
    """
    Args:
        wc : cavity frequency in GHz
        wa : qubit frequency in GHz
        N: order of fock state
    Return:
        tuple of eigenvalues and eigenvectors
    """
    a  = tensor(destroy(N), qeye(2))
    sm = tensor(qeye(N), destroy(2))
    nc = a.dag() * a
    na = sm.dag() * sm
    use_rwa = False

    if use_rwa: 
        H = wc * nc + wa * na + g * (a.dag() * sm + a * sm.dag())
    else:
        H = wc * nc + wa * na + g * (a.dag() + a) * (sm + sm.dag())

    # find the groundstate of the composite system
    evals, ekets = H.eigenstates()

    return evals, ekets

In [3]:
N = 10
wa = 5
g = 0.1
wc_list = np.linspace(4,6,101)

eigenvals = np.array([compute_energies_two_level_qubit(wc, wa, g, N=N)[0][0:3] for wc in wc_list])
E_10 = eigenvals[:,1] - eigenvals[:,0]
E_20 = eigenvals[:,2] - eigenvals[:,0]
diff = E_20 - E_10

In [5]:
fig, ax = plt.subplots(1,1, figsize=(8,6))
ax.plot(wc_list, E_10, wc_list, E_20)
ax.plot(wc_list, diff)
ax.set_xlabel(r'Cavity Frequency (GHz)', fontsize=18)
ax.set_ylabel(r'Frequency (GHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid()
ax.set_title(f'Two-level qubit + resonator\n Qubit Freq = {wa} GHz, g = {g} GHz', fontsize=18)
print(f'split = {np.min(diff):.3f} GHz')

split = 0.200 GHz


# Transmon + resonator

In [18]:
def compute_energies_transmon_resonator(Ec, Ej, g01, wc, N=10, ng=0, M_res=3):
    """
    Solve Hamiltoninian of transmon + resonator to compute eigenvalues/eigenvectors
    Args:
        Ec, Ej: Charging energy, Josephson energy in GHz
        g01 : 0-1 coupling strength in GHz, See (3.3) and (3.4) in Koch's paper
        wc: Cavity frequency in GHz
        N : number of change basis = 2N + 1
        M_res : order of fock state for cavity
    Return:
        tuple of eigenvalues and eigenvectors
    """
    
    a = destroy(M_res)
    
    H_tr = tensor(Qobj(np.diag(4 * Ec * (np.arange(-N,N+1)-ng)**2) \
        + 0.5 * Ej * (np.diag(-np.ones(2*N), 1) + np.diag(-np.ones(2*N), -1))), qeye(M_res))
    
    H_r = tensor(qeye(2*N+1), wc * a.dag() * a)
    H_int = g01/(np.power(Ej/Ec/8, 1/4)/np.sqrt(2)) *  tensor( Qobj(np.diag(np.arange(-N,N+1))), a.dag() + a)
    
    H = H_tr + H_r + H_int
    
    evals, ekets = H.eigenstates()

    return evals, ekets

def calculate_numerical_mat_element_01(Ec, Ej, N=10, ng=0):
    """ calculate <0|n|1>"""
    H = Qobj(np.diag(4 * Ec * (np.arange(-N,N+1)-ng)**2) 
                 + 0.5 * Ej * (np.diag(-np.ones(2*N), 1) + np.diag(-np.ones(2*N), -1)))
    eigenvals, eigenvecs = H.eigenstates()
    n_op = Qobj(np.diag((np.arange(-N,N+1)-ng))) 
    matrix_element_01 = n_op.matrix_element(eigenvecs[0], eigenvecs[1])
    return matrix_element_01

In [24]:
# plot energy diff vs wc
N, M_res = 10, 3
Ec, Ej = 0.35, 19.5
g01 = 0.05 

wc_list = compute_transmon_f01(Ej, Ec) + np.linspace(-g01*20 ,g01*20, 101)

eigenvals = np.array([compute_energies_transmon_resonator(Ec, Ej, g01, wc, N=N, ng=0, M_res=M_res)[0][0:3] for wc in wc_list])
E_10 = eigenvals[:,1] - eigenvals[:,0]  # energy difference between 0 and 1 
E_20 = eigenvals[:,2] - eigenvals[:,0]  # energy difference between 0 and 2 
diff = E_20 - E_10

fig, ax = plt.subplots(1,1, figsize=(8,6))
ax.plot(wc_list, E_10, wc_list, E_20)
ax.plot(wc_list, diff, label='diff')
ax.set_xlabel(r'Cavity Frequency (GHz)', fontsize=18)
ax.set_ylabel(r'Frequency (GHz)', fontsize=18)
ax.tick_params(axis='x', labelsize=18)
ax.tick_params(axis='y', labelsize=18)
ax.grid()
ax.legend(fontsize=18)
ax.set_title(f'Transmon + resonator\nEc = {Ec}, Ej = {Ej} GHz, f01 = {compute_transmon_f01(Ej, Ec):.3f} GHz, g01 = {g01} GHz', fontsize=18)
print(f'split = {np.min(diff):.3f} GHz')

split = 0.097 GHz


In [25]:
# g01 vs split comparion
N, M_res = 10, 3
Ec, Ej = 0.35, 19.5 # Ej = 15, 19.5, 25  --> qubit freq = 6.11, 7, 8 GHz
g01_list = np.linspace(0.005, 0.05, 11)
wc = compute_transmon_f01(Ej, Ec)

diff_min_list = []
for g in g01_list:
    f01 = compute_transmon_f01(Ej, Ec)
    wc_list = f01 + np.linspace(-g*2, g*2, 101)
    eigenvals = np.array([compute_energies_transmon_resonator(Ec, Ej, g, wc, N=N, ng=0, M_res=M_res)[0][0:3] for wc in wc_list])
    E_10 = eigenvals[:,1] - eigenvals[:,0]
    E_20 = eigenvals[:,2] - eigenvals[:,0]
    diff = E_20 - E_10
    diff_min_list.append(np.min(diff))

diff_min_list = np.array(diff_min_list)
g01_list = g01_list/ (np.power(Ej/Ec/8, 1/4)/np.sqrt(2))\
        *np.absolute(calculate_numerical_mat_element_01(Ec,Ej))

ratio_split_2g = np.round(diff_min_list/(2*g01_list), decimals=3)
avg_ratio_split_2g = np.mean(ratio_split_2g)

fig1, ax1 = plt.subplots(1,1,figsize=(8,6))
ax1.plot(g01_list,ratio_split_2g ,'r-o' )
ax1.grid()
ax1.tick_params(axis='x', labelsize=18)
ax1.tick_params(axis='y', labelsize=18)
ax1.set_xlabel(r'g (GHz)', fontsize=18)
ax1.legend([r'split/($2g_{01}$)'], fontsize=20)
ax1.set_ylim([avg_ratio_split_2g * 0.98, avg_ratio_split_2g*1.02])
ax1.set_title(f'Transmon + cavity\nEc = {Ec}, Ej = {Ej} GHz, Cavity Freq = f01 = {compute_transmon_f01(Ej, Ec):.3f} GHz', fontsize=18)

Text(0.5, 1.0, 'Transmon + cavity\nEc = 0.35, Ej = 19.5 GHz, Cavity Freq = f01 = 7.020 GHz')

In [20]:
Ec, Ej = 0.35,19
print(f'f01 = {compute_transmon_f01(Ej, Ec):.3f} GHz')
print(f'<0|n|1> = {np.power((Ej/Ec/8), 1/4)/np.sqrt(2)}')

f01 = 6.924 GHz
<0|n|1> = 1.141258156873071


# Find f01 and resonator frequency

In [21]:
def compute_transmon_resonator_frequency(Ec, Ej, g01, wc, N=10, ng=0, M_res=3):
    """
    Solve Hamiltoninian of transmon + resonator to compute eigenvalues/eigenvectors
    Args:
        Ec, Ej: Charging energy, Josephson energy in GHz
        g01 : 0-1 coupling strength in GHz, See (3.3) and (3.4) in Koch's paper
        wc: Cavity frequency in GHz
        N : number of change basis = 2N + 1
        M_res : order of fock state for cavity
    Return:
        tuple of qubit f01 and cavity frequency in GHz
    """
    
    eigenvals = compute_energies_transmon_resonator(Ec, Ej, g01, wc, N=N, ng=0, M_res=M_res)[0][0:3]
    E_10 = eigenvals[1] - eigenvals[0]
    E_20 = eigenvals[2] - eigenvals[0]
        
    return E_10, E_20

In [23]:
Ec, Ej = 0.35, 19.5
wc = 7 
g01 = 0.05
compute_transmon_resonator_frequency(Ec, Ej, g01, wc) # one corresponds to qubit f01 and the other to cavity

(6.960129899714548, 7.05963479821)

# Compare $\langle 0|\hat{n}|1\rangle$ and $(E_j/E_c/8)^{1/4}/\sqrt(2)$

In [16]:
Ec, Ej = 0.35, 19.5
print(f'f01 = {compute_transmon_f01(Ej,Ec):.3f} GHz')
print(f'<0|n|1> = {calculate_numerical_mat_element_01(Ec,Ej).real:.3f}')
print(f'(Ej/Ec/8)^(1/4)/sqrt(2) = {np.power(Ej/Ec/8, 1/4)/np.sqrt(2):.3f}')

f01 = 7.020 GHz
<0|n|1> = -1.119
(Ej/Ec/8)^(1/4)/sqrt(2) = 1.149
